
# Week 4 — Clustering as a Functor (Single-Linkage on 1‑Lipschitz Maps)

**Goal.** Show a tiny, runnable instance of *functorial clustering*: from the category **FMS** of finite metric spaces with **1‑Lipschitz** maps (non‑expansive maps) to the category **Ult** of ultrametric spaces (dendrograms), via **single‑linkage hierarchical clustering**.

We’ll compute the single‑linkage **ultrametric** \(u_X\) from a metric space \((X,d_X)\). Then, given a 1‑Lipschitz map \(f: X \to Y\), we’ll verify the functoriality inequality
\[
u_Y(f(x), f(x')) \;\le\; u_X(x,x') \quad \text{for all } x,x' \in X,
\]
which induces compatible maps between partitions at every threshold \(t\) (naturality across the dendrogram).


## Environment (run first) — CPU‑only, quick installs

In [ ]:

%%capture
%pip install -q numpy==1.* networkx==3.* matplotlib==3.*


## Helper functions: metric → single‑linkage ultrametric via MST

In [ ]:

import numpy as np, networkx as nx
from math import sqrt

def euclid(p, q):
    return sqrt((p[0]-q[0])**2 + (p[1]-q[1])**2)

def pairwise_d(points):
    keys = list(points.keys())
    D = {(a,b): euclid(points[a], points[b]) for i,a in enumerate(keys) for b in keys[i+1:]}
    # Symmetrize and zeros
    for a in keys:
        D[(a,a)] = 0.0
        for b in keys:
            if (a,b) not in D and (b,a) in D:
                D[(a,b)] = D[(b,a)]
    return D

def single_linkage_ultrametric(points):
    """Return ultrametric u: dict[(i,j)] -> merge height via MST max-edge on the path."""
    keys = list(points.keys())
    D = pairwise_d(points)
    G = nx.Graph()
    for a in keys:
        G.add_node(a)
    # Complete graph
    for i,a in enumerate(keys):
        for b in keys[i+1:]:
            G.add_edge(a,b, weight=D[(a,b)])
    T = nx.minimum_spanning_tree(G, weight='weight')
    # Precompute all-pairs path max-edge on MST
    def path_max(a,b):
        path = nx.shortest_path(T, a, b)  # unique path in tree
        mw = 0.0
        for u,v in zip(path, path[1:]):
            w = T[u][v]['weight']
            if w > mw: mw = w
        return mw
    U = {}
    for i,a in enumerate(keys):
        for b in keys[i+1:]:
            U[(a,b)] = path_max(a,b)
            U[(b,a)] = U[(a,b)]
        U[(a,a)] = 0.0
    return U

def clusters_from_ultrametric(U, threshold):
    elems = sorted({a for (a,_) in U.keys()})
    parent = {e:e for e in elems}
    def find(x):
        while parent[x]!=x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(a,b):
        ra, rb = find(a), find(b)
        if ra != rb: parent[rb] = ra
    for (a,b),uv in U.items():
        if a<b and uv <= threshold:
            union(a,b)
    clusters = {}
    for e in elems:
        clusters.setdefault(find(e), set()).add(e)
    return list(clusters.values())


## Construct two metric spaces and a 1‑Lipschitz map \(f:X\to Y\)

In [ ]:

# X: a small point cloud in R^2
X = {
    "a": (0.0, 0.0),
    "b": (1.0, 0.0),
    "c": (0.2, 1.0),
    "d": (2.0, 0.1),
}

# Build Y by scaling coordinates by 0.5 (identity-on-labels map becomes non-expansive)
scale = 0.5
Y = {k: (scale*v[0], scale*v[1]) for k,v in X.items()}

def f(label_in_X):
    return label_in_X  # identity on labels

# Compute ultrametrics
U_X = single_linkage_ultrametric(X)
U_Y = single_linkage_ultrametric(Y)

# Verify u_Y(f(x),f(x')) <= u_X(x,x')
labels = sorted(X.keys())
ok = True
violations = []
for i,a in enumerate(labels):
    for b in labels[i+1:]:
        lhs = U_Y[(f(a), f(b))]
        rhs = U_X[(a,b)]
        if lhs > rhs + 1e-9:
            ok = False
            violations.append((a,b,lhs,rhs))
ok, violations[:3]



**Expected:** `ok` is `True` (no violations).  
Because the map is 1‑Lipschitz, merge heights in the image (Y) are no larger than in the source (X).


## Visualize partitions at a few thresholds

In [ ]:

for t in [0.3, 0.6, 1.0]:
    Cx = clusters_from_ultrametric(U_X, t)
    Cy = clusters_from_ultrametric(U_Y, t)
    print(f"t={t:0.1f}")
    print("  X clusters:", [sorted(s) for s in Cx])
    print("  Y clusters:", [sorted(s) for s in Cy])


## Negative test: expansive map (not 1‑Lipschitz) can violate the inequality

In [ ]:

scale_bad = 1.6   # expands distances
Y_bad = {k: (scale_bad*v[0], scale_bad*v[1]) for k,v in X.items()}
U_Y_bad = single_linkage_ultrametric(Y_bad)

viol = []
for i,a in enumerate(labels):
    for b in labels[i+1:]:
        if U_Y_bad[(a,b)] > U_X[(a,b)] + 1e-9:
            viol.append((a,b,U_Y_bad[(a,b)],U_X[(a,b)]))
print("Number of violations with expansive map:", len(viol))
print("First few:", viol[:5])


## (Optional) Quick plot of the two point clouds

In [ ]:

import matplotlib.pyplot as plt

fig = plt.figure()
xs = [X[k][0] for k in labels]; ys = [X[k][1] for k in labels]
plt.scatter(xs, ys)
for k in labels:
    plt.text(X[k][0]+0.02, X[k][1]+0.02, k)
plt.title("Point cloud X (R^2)"); plt.xlabel("x"); plt.ylabel("y"); plt.show()

fig = plt.figure()
ys_x = [Y[k][0] for k in labels]; ys_y = [Y[k][1] for k in labels]
plt.scatter(ys_x, ys_y)
for k in labels:
    plt.text(Y[k][0]+0.02, Y[k][1]+0.02, k)
plt.title("Point cloud Y = scaled(X)"); plt.xlabel("x"); plt.ylabel("y"); plt.show()



## Exercises
1. **Change the map.** Replace the scaling with a different 1‑Lipschitz map (e.g., a projection, or clamp coordinates) and re‑run the inequality check.
2. **Compare linkage methods.** Swap single‑linkage for complete‑linkage and see that the functorial inequality may fail for non‑expansive maps.
3. **From ultrametrics to dendrograms.** Extend `clusters_from_ultrametric` to build merge trees (a dendrogram), and verify that \(f\) induces a morphism of dendrograms for all thresholds.
4. **Kleinberg vs. functoriality (discussion).** Why does Kleinberg’s axioms yield an impossibility for flat clustering, while hierarchical + functoriality yields existence/uniqueness (single‑linkage w.r.t. 1‑Lipschitz morphisms)?
